In [1]:
# !git clone -b master https://github.com/StefanoNasini/Country-by-country-2022-2023-international-relationships
# !cd Country-by-country-2022-2023-international-relationships

In [2]:
import os
root = os.getcwd()
root = os.path.dirname(root)
import pandas as pd

friends90_tri = pd.read_csv(os.path.join(root, "constructed_network", "final_network", "friends90_tri.csv"), index_col=0)
prerequisite = pd.read_csv(os.path.join(root, "constructed_network", "prerequisite_4items.csv"), index_col=0)


UN_tri = friends90_tri.columns

os.chdir(os.path.join(root, 'src', 'utils'))
from aka import tri2aka, tri2name, name2tri, guess_country
os.chdir(root)
os.chdir("src")

['Y AFGHANISTAN\n', 'A ALBANIA\n', 'Y ALGERIA\n', 'A ANDORRA\n', 'Y ANGOLA\n', 'Y ANTIGUA AND BARBUDA\n', 'Y ARGENTINA\n', 'A ARMENIA\n', 'N AUSTRALIA\n', 'N AUSTRIA\n']


In [3]:
import itertools
import subprocess


parameter_combinations = []

        
# Define the parameter values
rho_values = [0.1, 0.5, 1]
include_NE_values = [0, 1]
include_LI_values = [0]
include_sym_values = [0] # Labeling symmetry
theta_shrink_values = [1, 0.7, 0.8, 0.9]
phi_data_values = [os.path.join(root, "constructed_network","phidata_network80.txt"), 
                   os.path.join(root, "constructed_network", "phidata_network90.txt")]
relax_values = [0, 1]
 
# Define the base command
base_command = f'"{os.path.join(root, "Project1.exe")}" 184 4 4'
output_name = "UN_application.txt"
 
# Iterate through the parameter combinations with the requested order of for loops

for relax in relax_values:
    for theta_shrink in theta_shrink_values:
        for include_NE in include_NE_values:
            for include_LI in include_LI_values:
                for phi_data in phi_data_values:
                    for rho in rho_values:
                        for include_sym in include_sym_values:
                            parameter_combinations.append((rho, include_NE, include_LI, include_sym, theta_shrink, phi_data, relax))


In [7]:
os.chdir(root)
if "result" not in os.listdir():
    os.mkdir("result")
os.chdir(os.path.join(root, "result"))

acc=0
for (rho, include_NE, include_LI, include_sym, theta_shrink, phi_data, relax) in parameter_combinations:
    v_predefined_data = os.path.join(root, "constructed_network", "predefined.txt")
    w_data = os.path.join(root, "constructed_network", "w_data_real_double.txt")
    command = fr'{base_command} {rho} {include_NE} {include_LI} {include_sym} {theta_shrink} "{phi_data}" "{v_predefined_data}" "{w_data}" {relax} 60 0 8 0 {output_name}'
    print(command)
    
    result = subprocess.run(command, capture_output=True)
    print('-------------', acc, '--------------')
    acc+=1
os.chdir(os.path.join(root, "src"))

"C:\Users\y.lin\OneDrive - IESEG\Desktop\Data\Data\Project1.exe" 184 4 4 0.1 0 0 0 1 "C:\Users\y.lin\OneDrive - IESEG\Desktop\Data\Data\constructed_network\phidata_network80.txt" "C:\Users\y.lin\OneDrive - IESEG\Desktop\Data\Data\constructed_network\predefined.txt" "C:\Users\y.lin\OneDrive - IESEG\Desktop\Data\Data\constructed_network\w_data_real_double.txt" 0 60 0 8 0 UN_application.txt
------------- 0 --------------
"C:\Users\y.lin\OneDrive - IESEG\Desktop\Data\Data\Project1.exe" 184 4 4 0.5 0 0 0 1 "C:\Users\y.lin\OneDrive - IESEG\Desktop\Data\Data\constructed_network\phidata_network80.txt" "C:\Users\y.lin\OneDrive - IESEG\Desktop\Data\Data\constructed_network\predefined.txt" "C:\Users\y.lin\OneDrive - IESEG\Desktop\Data\Data\constructed_network\w_data_real_double.txt" 0 60 0 8 0 UN_application.txt
------------- 1 --------------
"C:\Users\y.lin\OneDrive - IESEG\Desktop\Data\Data\Project1.exe" 184 4 4 1 0 0 0 1 "C:\Users\y.lin\OneDrive - IESEG\Desktop\Data\Data\constructed_network\ph

KeyboardInterrupt: 

In [ ]:
# read the run result, the choices suggested by the model

v = pd.read_csv(os.path.join(root, "result", "UN_application.txt"), names=range(737))
v = v.drop(columns=[736])
v = v>0.5


# read the output_tables having detailed running information of the program

columns=["rho", "include_NE", "include_LI", "include_sym", "theta_shrink", "phi_file", 
         "LP_relax", "lpstat", "objective", "sum_eta", "solution_time","node_count", "tolerance_optimality",
        "n_cols", "n_rows"]
ot = pd.read_csv(os.path.join(root, "result", "output_table.txt"), sep="\t", names=columns, index_col=False)
ot

In [ ]:
# reading real choice (built from the data) and the significant choice (built from the data)
# they are useful in evaluating the solution

v_real = pd.read_csv(os.path.join(root, "constructed_network", "v_real_choice_4items.csv"), index_col=0)
v_real = v_real.astype(bool)
v_real

significant_choice = pd.read_csv(os.path.join(root, "constructed_network", "significant_choice_4items.csv"), index_col=0)
significant_choice70 = pd.read_csv(os.path.join(root, "constructed_network", "significant_choice70_4items.csv"), index_col=0)
significant_choice80 = pd.read_csv(os.path.join(root, "constructed_network", "significant_choice80_4items.csv"), index_col=0)
significant_choice85 = pd.read_csv(os.path.join(root, "constructed_network", "significant_choice85_4items.csv"), index_col=0)


# make LGBT only evaluate on 0-real-labeled countries

def remove_one_label(significant_choice):
    significant_choice_ = significant_choice.copy()
    for tri in UN_tri:
        if v_real.loc[tri, "LGBT"] == 0: # only consider 0-real-valued countries
            significant_choice_.loc[tri, "LGBT"] = True
        if prerequisite.loc[tri, "LGBT"] == 1: 
            significant_choice_.loc[tri, "LGBT"] = False
        if prerequisite.loc[tri, "LGBT"] == 0: # but not in prerequisite because it doesn't need to be evaluated
            significant_choice_.loc[tri, "LGBT"] = False
#             pass
    return significant_choice_

def remove_prerequisite(significant_choice):
    significant_choice_ = significant_choice.copy()
    for tri in UN_tri:
        for item in ['LGBT', 'environmental', 'migration']:

            if prerequisite.loc[tri, item] in [0, 1]: 
                significant_choice_.loc[tri, item] = False
#             pass
    return significant_choice_

significant_choice = remove_one_label(significant_choice)
significant_choice = remove_prerequisite(significant_choice)

significant_choice70 = remove_one_label(significant_choice70)
significant_choice70 = remove_prerequisite(significant_choice70)

significant_choice80 = remove_one_label(significant_choice80)
significant_choice80 = remove_prerequisite(significant_choice80)

significant_choice85 = remove_one_label(significant_choice85)
significant_choice85 = remove_prerequisite(significant_choice85)

In [ ]:
# evaluate the score of each item. For each item, select individuals that are significant

# set variables for evaluating each item separately 


# in order to compare the real value to the values of each of 96 settings, 
# we copy the real value 96 times so that the comparasion happens in the same scope

def copy_96times(df):
    df = pd.concat([df]*96, axis=1, ignore_index=True)
    df = df.transpose()
    return df

v_real_LGBT = copy_96times(v_real['LGBT'])
v_real_environmental = copy_96times(v_real['environmental'])
v_real_migration = copy_96times(v_real['migration'])

v_LGBT = v.iloc[:, range(0, 736, 4)]
v_LGBT.columns = v_real.index
v_environmental = v.iloc[:, range(1, 736, 4)]
v_environmental.columns = v_real.index
v_migration = v.iloc[:, range(2, 736, 4)]
v_migration.columns = v_real.index

In [ ]:
def evaluate(v_item, v_real_item, significant_choice, item_name):
    compared = (v_item==v_real_item)
    v_significant = significant_choice[item_name]>0.5
    compared = compared.loc[:, v_significant]
    ratio = compared.sum(axis=1) / compared.shape[1]
    return ratio
    #     print(ratio)

ot['LGBT'] = evaluate(v_migration, v_real_migration, significant_choice80, "LGBT")
ot['environmental'] = evaluate(v_migration, v_real_migration, significant_choice80, "environmental")
ot['migration'] = evaluate(v_migration, v_real_migration, significant_choice80, "migration")


ot.replace(os.path.join(root, "constructed_network","phidata_network80.txt"), "80", inplace=True)
ot.replace(os.path.join(root, "constructed_network","phidata_network90.txt"), "90", inplace=True)


In [ ]:
otover = ot[['theta_shrink','phi_file', 'rho', 'sum_eta', 'solution_time']]


In [ ]:
otover

In [ ]:
# make two columns for NE

for i in range(48):
    otover.loc[i, 'LR'] = ot.loc[i+48, 'objective']

for i in range(48):
    if i%12 in [0, 1, 2, 3, 4, 5]:
        otover.loc[i, 'sum_eta_NE'] = otover.loc[i+6, 'sum_eta']
        otover.loc[i, 'solution_time_NE'] = otover.loc[i+6, 'solution_time']
        otover.loc[i, 'LR_NE'] = ot.loc[i+6+48, 'objective']

otover

In [ ]:
otover = otover.dropna(axis='index', how='any')
otover = otover.round(decimals=2)
otover['sum_eta_NE'] = otover['sum_eta_NE'].map(lambda x: int(x))

In [ ]:
s = otover.to_csv(index=False, header=False).replace(',', ' & ').replace('\r\n', '\\\\\r\n')
lines = s.split('\r\n')
i = 0
for l in lines:
    print(l)
    i+=1
    if i%6 == 0:
        print('\hline')

In [ ]:
otne = ot[[ 'theta_shrink','phi_file', 'rho', 
        'LGBT', 'environmental', 'migration']]

In [ ]:
for i in range(96):
    if i%12 in [0, 1, 2, 3, 4, 5]:
        otne.loc[i, 'LGBT_NE'] = otne.loc[i+6, 'LGBT']
        otne.loc[i, 'environmental_NE'] = otne.loc[i+6, 'environmental']
        otne.loc[i, 'migration_NE'] = otne.loc[i+6, 'migration']
#         otne.loc[i, 'item4_NE'] = otne.loc[i+6, 'item4']

In [ ]:
otne = otne.dropna(axis='index', how='any')

In [ ]:
otne['improvement'] = None
otne['improvement'] = 0 + (otne['LGBT_NE'] - otne['LGBT'] > 0) + (otne['environmental_NE'] - otne['environmental'] > 0) +  (otne['migration_NE'] - otne['migration'] > 0) 
otne['improvement'] = otne['improvement'].map(lambda x : '*'*x)


In [ ]:
otne = otne.round(decimals=2)
otne = otne.iloc[:24, :]

In [ ]:
s = otne.to_csv(index=False, header=False).replace(',', ' & ').replace('\r\n', '\\\\\r\n')
lines = s.split('\r\n')
i = 0
for l in lines:
    print(l)
    i+=1
    if i%6 == 0:
        print('\hline')